Starting from an address in a shopping center in the area, Walmart store in this case, Mr. Alex wants a report about surrounding businesses and any other information that may be useful for him to take an educated decision. In this first part of the project I will be just getting the latitude and longitude coordinates of the chosen address. Next part, I will be exploring the neighborhood to discover multiple venues, and do any necessary transformations such as filtering and converting to a pandas dataframe for processing.

<h2>Imporitng necessary packages</h2>

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

### Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'PR2FLHCYDFMV0GMWWXCYWUBZSQZ3Y5J4VEZNNY04H3MGIAQY' # your Foursquare ID
CLIENT_SECRET = 'TMXGSTMOXVULRNANYXHW2TPCFVU2SXJUIUHMUGH5EC4R0ARC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PR2FLHCYDFMV0GMWWXCYWUBZSQZ3Y5J4VEZNNY04H3MGIAQY
CLIENT_SECRET:TMXGSTMOXVULRNANYXHW2TPCFVU2SXJUIUHMUGH5EC4R0ARC


#### I assume that I am starting from an address in a shopping center, I am using Walmart store address. So let's start by converting the Walmart's address to its latitude and longitude coordinates.

In [4]:
address = '1755 Hilliard Rome Rd, Hilliard, Ohio, 43026'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.013429 -83.153332


### 1. Search for a specific venue category

In [5]:
search_query = 'restaurant'
radius = 5000
print(search_query + ' .... OK!')

restaurant .... OK!


#### Define the corresponding URL

In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PR2FLHCYDFMV0GMWWXCYWUBZSQZ3Y5J4VEZNNY04H3MGIAQY&client_secret=TMXGSTMOXVULRNANYXHW2TPCFVU2SXJUIUHMUGH5EC4R0ARC&ll=40.013429,-83.153332&v=20180604&query=restaurant&radius=5000&limit=50'

#### Send the GET Request and examine the results

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cfba9a49fb6b77580b28f6a'},
 'response': {'venues': [{'id': '4dbd74d36a23e294ba458425',
    'name': 'Restaurant De Candy',
    'location': {'lat': 40.00291201,
     'lng': -83.1598368,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.00291201,
       'lng': -83.1598368}],
     'distance': 1295,
     'postalCode': '43026',
     'cc': 'US',
     'city': 'Hilliard',
     'state': 'OH',
     'country': 'United States',
     'formattedAddress': ['Hilliard, OH 43026', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d10e941735',
      'name': 'Greek Restaurant',
      'pluralName': 'Greek Restaurants',
      'shortName': 'Greek',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/greek_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1559996836',
    'hasPerk': False},
   {'id': '4f32b12319836c91c7f05f05',
    'name': 'Number One Chinese Restaurant',
    'location': {'address': '5429 Rob

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(100)

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,4dbd74d36a23e294ba458425,NaN,...,1295,"[Hilliard, OH 43026, United States]","[{'label': 'display', 'lat': 40.00291201, 'lng...",40.002912,-83.159837,43026,OH,Restaurant De Candy,v-1559996836,NaN
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f32b12319836c91c7f05f05,5429 Roberts Rd,...,1084,"[5429 Roberts Rd, Hilliard, OH 43026, United S...","[{'label': 'display', 'lat': 40.003719, 'lng':...",40.003719,-83.152254,43026,OH,Number One Chinese Restaurant,v-1559996836,NaN
2,"[{'id': '4bf58dd8d48988d154941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b61f144f964a520802c2ae3,4453 Cemetery Rd,...,2561,"[4453 Cemetery Rd (Lacon Rd), Hilliard, OH 430...","[{'label': 'display', 'lat': 40.03189453676746...",40.031895,-83.135396,43026,OH,Louie's Grill Fusion Restaurant,v-1559996836,NaN
3,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b593184f964a520d68028e3,1451 Rome Hilliard Rd,...,3755,"[1451 Rome Hilliard Rd, Columbus, OH 43228, Un...","[{'label': 'display', 'lat': 39.97972627011028...",39.979726,-83.151348,43228,OH,Perkins Restaurant & Bakery,v-1559996836,NaN
4,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",487912,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/bob-evans-1...,False,4ce71b5fe1eeb60cc6f49aae,1111 Rome Hilliard Rd,...,4307,"[1111 Rome Hilliard Rd (Feder Rd), Columbus, O...","[{'label': 'display', 'lat': 39.97476935714648...",39.974769,-83.151108,43228,OH,Bob Evans Restaurant,v-1559996836,NaN
5,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",NaN,NaN,NaN,NaN,NaN,NaN,False,5421d1c9498e1b6b9c196487,3901 Britton Pkwy,...,2827,"[3901 Britton Pkwy, Hilliard, OH 43026, United...","[{'label': 'display', 'lat': 40.03281814075288...",40.032818,-83.131910,43026,OH,Rusty Bucket Restaurant and Tavern,v-1559996836,NaN
6,"[{'id': '4bf58dd8d48988d1df931735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f324d7019836c91c7ca5a37,5240 Cemetery Rd,...,1877,"[5240 Cemetery Rd, Hilliard, OH 43026, United ...","[{'label': 'display', 'lat': 40.030232, 'lng':...",40.030232,-83.155238,43026,OH,Galaxy Restaurant,v-1559996836,NaN
7,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c6b2ce89c76d13a48f84d0f,5344 Center St,...,2355,"[5344 Center St (Main Street), Hilliard, OH 43...","[{'label': 'display', 'lat': 40.03402818456824...",40.034028,-83.159642,43026,OH,Otie's Restaurant & Pub,v-1559996836,NaN
8,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3284ee19836c91c7dfb9d6,5286 Center St,...,2209,"[5286 Center St, Hilliard, OH 43026, United St...","[{'label': 'display', 'lat': 40.03293, 'lng': ...",40.032930,-83.158160,43026,OH,Benito's Restaurant Bar and Grill,v-1559996836,NaN
9,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b0d2edaf964a520634423e3,3800 Fishinger Blvd.,...,3897,[3800 Fishinger Blvd. (at Park Mill Run Drive)...,"[{'label': 'display', 'lat': 40.031346, 'lng':...",40.031346,-83.114056,43026,OH,First Watch,v-1559996836,NaN


#### Define information of interest and filter dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Restaurant De Candy,Greek Restaurant,NaN,US,Hilliard,United States,NaN,1295,"[Hilliard, OH 43026, United States]","[{'label': 'display', 'lat': 40.00291201, 'lng...",40.002912,-83.159837,43026,OH,4dbd74d36a23e294ba458425
1,Number One Chinese Restaurant,Chinese Restaurant,5429 Roberts Rd,US,Hilliard,United States,NaN,1084,"[5429 Roberts Rd, Hilliard, OH 43026, United S...","[{'label': 'display', 'lat': 40.003719, 'lng':...",40.003719,-83.152254,43026,OH,4f32b12319836c91c7f05f05
2,Louie's Grill Fusion Restaurant,Cuban Restaurant,4453 Cemetery Rd,US,Hilliard,United States,Lacon Rd,2561,"[4453 Cemetery Rd (Lacon Rd), Hilliard, OH 430...","[{'label': 'display', 'lat': 40.03189453676746...",40.031895,-83.135396,43026,OH,4b61f144f964a520802c2ae3
3,Perkins Restaurant & Bakery,American Restaurant,1451 Rome Hilliard Rd,US,Columbus,United States,NaN,3755,"[1451 Rome Hilliard Rd, Columbus, OH 43228, Un...","[{'label': 'display', 'lat': 39.97972627011028...",39.979726,-83.151348,43228,OH,4b593184f964a520d68028e3
4,Bob Evans Restaurant,American Restaurant,1111 Rome Hilliard Rd,US,Columbus,United States,Feder Rd,4307,"[1111 Rome Hilliard Rd (Feder Rd), Columbus, O...","[{'label': 'display', 'lat': 39.97476935714648...",39.974769,-83.151108,43228,OH,4ce71b5fe1eeb60cc6f49aae
5,Rusty Bucket Restaurant and Tavern,Restaurant,3901 Britton Pkwy,US,Hilliard,United States,NaN,2827,"[3901 Britton Pkwy, Hilliard, OH 43026, United...","[{'label': 'display', 'lat': 40.03281814075288...",40.032818,-83.131910,43026,OH,5421d1c9498e1b6b9c196487
6,Galaxy Restaurant,BBQ Joint,5240 Cemetery Rd,US,Hilliard,United States,NaN,1877,"[5240 Cemetery Rd, Hilliard, OH 43026, United ...","[{'label': 'display', 'lat': 40.030232, 'lng':...",40.030232,-83.155238,43026,OH,4f324d7019836c91c7ca5a37
7,Otie's Restaurant & Pub,Bar,5344 Center St,US,Hilliard,United States,Main Street,2355,"[5344 Center St (Main Street), Hilliard, OH 43...","[{'label': 'display', 'lat': 40.03402818456824...",40.034028,-83.159642,43026,OH,4c6b2ce89c76d13a48f84d0f
8,Benito's Restaurant Bar and Grill,Pizza Place,5286 Center St,US,Hilliard,United States,NaN,2209,"[5286 Center St, Hilliard, OH 43026, United St...","[{'label': 'display', 'lat': 40.03293, 'lng': ...",40.032930,-83.158160,43026,OH,4f3284ee19836c91c7dfb9d6
9,First Watch,Breakfast Spot,3800 Fishinger Blvd.,US,Columbus,United States,at Park Mill Run Drive,3897,[3800 Fishinger Blvd. (at Park Mill Run Drive)...,"[{'label': 'display', 'lat': 40.031346, 'lng':...",40.031346,-83.114056,43026,OH,4b0d2edaf964a520634423e3


#### Let's visualize the restaurants that are nearby

In [10]:
dataframe_filtered.name

0                    Restaurant De Candy
1          Number One Chinese Restaurant
2        Louie's Grill Fusion Restaurant
3            Perkins Restaurant & Bakery
4                   Bob Evans Restaurant
5     Rusty Bucket Restaurant and Tavern
6                      Galaxy Restaurant
7                Otie's Restaurant & Pub
8      Benito's Restaurant Bar and Grill
9                            First Watch
10                             Bob Evans
11         Kyo Ichiban Buffet Restaurant
12                    Dang ho Restaurant
13          Toro Loco Mexican Restaurant
14       Montelongo's Mexican Restaurant
15     Louie's Grill & Fusion Restaurant
16                           Grand China
17                            El Vaquero
Name: name, dtype: object

In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent El Vaquero restaurant
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Restaurant De Candy',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [12]:
dataframe_filtered['id']

0     4dbd74d36a23e294ba458425
1     4f32b12319836c91c7f05f05
2     4b61f144f964a520802c2ae3
3     4b593184f964a520d68028e3
4     4ce71b5fe1eeb60cc6f49aae
5     5421d1c9498e1b6b9c196487
6     4f324d7019836c91c7ca5a37
7     4c6b2ce89c76d13a48f84d0f
8     4f3284ee19836c91c7dfb9d6
9     4b0d2edaf964a520634423e3
10    4b254d48f964a5209c6f24e3
11    50f8a751e4b013665a7060ed
12    4faaf9e9e4b034d5343592a2
13    4f4d620be4b081be1d37a65b
14    5627c42f498e296dba6434b1
15    4c027853f423a59331c8ce16
16    4b22e2a8f964a5207e4f24e3
17    4b5495baf964a5203ac127e3
Name: id, dtype: object

#### Let's explore the rating of each of the venues 

In [13]:
for venue_id in dataframe_filtered['id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    url
    result = requests.get(url).json()
    #print(result['response']['venue'].keys())
    #result['response']['venue']
    try:
        #print(dataframe_filtered['name'])
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

This venue has not been rated yet.
This venue has not been rated yet.
7.7
7.4
7.3
7.6
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
7.8
6.2
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
8.0
8.0


In [24]:
venue_id = '4b254d48f964a5209c6f24e3' # ID of Bob evans with the lowest rating
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4b254d48f964a5209c6f24e3?client_id=PR2FLHCYDFMV0GMWWXCYWUBZSQZ3Y5J4VEZNNY04H3MGIAQY&client_secret=TMXGSTMOXVULRNANYXHW2TPCFVU2SXJUIUHMUGH5EC4R0ARC&v=20180604'

In [25]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4b254d48f964a5209c6f24e3',
 'name': 'Bob Evans',
 'contact': {'phone': '6147716799',
  'formattedPhone': '(614) 771-6799',
  'twitter': 'bobevansfarms',
  'instagram': 'bobevansfarms',
  'facebook': '111691362202833',
  'facebookUsername': 'BobEvansHilliard',
  'facebookName': 'Bob Evans'},
 'location': {'address': '3910 Lyman Dr',
  'crossStreet': 'at Cemetery Rd.',
  'lat': 40.0332014043564,
  'lng': -83.12815636396408,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.0332014043564,
    'lng': -83.12815636396408}],
  'postalCode': '43026',
  'cc': 'US',
  'city': 'Hilliard',
  'state': 'OH',
  'country': 'United States',
  'formattedAddress': ['3910 Lyman Dr (at Cemetery Rd.)',
   'Hilliard, OH 43026',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/bob-evans/4b254d48f964a5209c6f24e3',
 'categories': [{'id': '4bf58dd8d48988d14e941735',
   'name': 'American Restaurant',
   'pluralName': 'American Restaurants',
   'shortName': 'American',
   'icon': {'pr

### Confirm the venue's overall rating

In [26]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.2


### Get the number of tips

In [27]:
result['response']['venue']['tips']['count']

12

In [28]:
venue_id = '4b5495baf964a5203ac127e3' # ID of Elvaqeuro one of the two with the highest rating
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    
result['response']['venue']['tips']['count']   

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])
8.0


32

### Explore the neighborhood of Bob evans restaurant

In [38]:
lat = 40.033201 
lng =  -83.128156
limit = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
#url
results = requests.get(url).json()
results
'There are {} around Bob evans restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 2 around Bob evans restaurant.'

### Check the number of tips of one of the highest rated venues